<a href="https://colab.research.google.com/github/aovolkov/sentiment_analysis/blob/main/notebooks/twitter_sentiment_analysis_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определение тональности твитов с помощью BERT

## Установка библиотек

In [ ]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 9.5MB/s 
     |████████████████████████████████| 890kB 14.2MB/s 
     |████████████████████████████████| 1.1MB 25.6MB/s 
     |████████████████████████████████| 133kB 39.6MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 7.0MB 37.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=26d2ddbed064077108b3823fca7a50909b5ed162e5010f93bbcce32e0ec5a532
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.28 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import AdamW, BertForSequenceClassification
from IPython.display import clear_output
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    print('Device is cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))

Tesla T4


## Загрузка данных


Для данной задачи был выбран датасет с разметкой сентимента русскоязычных твитов (подробнее про него в [статье](http://www.swsys.ru/index.php?page=article&id=3962&lang=)). Корпус твитов содержит 114,911 положительных (класс 1) и 111,923 отрицательных (класс 0) записей. Загрузить его можно [тут](https://study.mokoron.com/).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
negative_texts = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Sentiment/negative_twitter.csv', encoding='utf8', sep=';', header=None)
positive_texts = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Sentiment/positive_twitter.csv', encoding='utf8', sep=';', header=None)

In [ ]:
positive_texts.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11
100349,411080979726757888,1386844317,rusekadopypi,"надену купальник слегка накрашу глаза, веть на...",1,0,0,0,537,191,212,0
12132,409308978607562752,1386421839,fyjofajanaw,"Вот убираешься в шкафу и находишь шортики, кот...",1,0,0,0,459,138,144,0
56519,410091517101883392,1386608411,ValeriaLomteva,Завтра последняя олимпиада в этом учебном году...,1,0,0,0,21965,311,249,9
95854,411018975297146880,1386829534,ariwkapsi,"@anair1991 я думала ты тут чаще бываешь,чем вк...",1,0,0,0,1768,4,8,1
1683,408924523367374849,1386330178,indigo_ilya,"Кстати, вчерашний пример идиотизма) http://t.c...",1,0,0,0,9182,193,269,3


In [ ]:
sentences = np.concatenate([positive_texts[3].values, negative_texts[3].values])

sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = [[1] for _ in range(positive_texts.shape[0])] + [[0] for _ in range(negative_texts.shape[0])]

# проверка на длину 
assert len(sentences) == len(labels) == positive_texts.shape[0] + negative_texts.shape[0]

In [ ]:
print(sentences[1000])

[CLS] Дим, ты помогаешь мне, я тебе, все взаимно, все правильно) [SEP]


In [ ]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.25)

print(len(train_labels), len(test_labels))

170125 56709


## Предобработка входных данных

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
print(tokenized_texts[2])

100%|██████████| 231508/231508 [00:00<00:00, 1075958.55B/s]


['[CLS]', 'к', '##а', '##к', 'о', '##ка', '##з', '##а', '##л', '##о', '##с', '##ь', ',', 'м', '##о', '##и', 'п', '##е', '##р', '##ч', '##а', '##т', '##к', '##и', 'р', '##а', '##б', '##о', '##т', '##а', '##ю', '##т', 'с', 'е', '##м', '##к', '##о', '##с', '##т', '##н', '##ы', '##м', 'с', '##е', '##н', '##с', '##о', '##р', '##о', '##м', 'o', '_', 'o', 'д', '##л', '##я', 'н', '##а', '##б', '##о', '##р', '##а', 'с', 'к', '##л', '##а', '##в', '##и', '##а', '##т', '##у', '##р', '##ы', 'н', '##е', 'п', '##о', '##д', '##х', '##о', '##д', '##я', '##т', ',', 'н', '##о', 'л', '##и', '##с', '##т', '##а', '##т', '##ь', 'л', '##е', '##н', '##т', '##у', 'о', '##ч', '##е', '##н', '##ь', 'д', '##а', '##ж', '##е', 'н', '##и', '##ч', '##е', '##г', '##о', '.', '[SEP]']


In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(
    input_ids,
    maxlen=100,
    dtype="long",
    truncating="post",
    padding="post"
)
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, train_gt, 
    random_state=42,
    test_size=0.1
)

train_masks, validation_masks, _, _ = train_test_split(
    attention_masks,
    input_ids,
    random_state=42,
    test_size=0.1
)

In [ ]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

In [ ]:
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
train_labels

In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(
    train_data,
    sampler=RandomSampler(train_data),
    batch_size=32
)

In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(
    validation_data,
    sampler=SequentialSampler(validation_data),
    batch_size=32
)

## Обучение модели

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
model.cuda()

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                                 'weight_decay_rate': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                                 'weight_decay_rate': 0.0}]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)



In [ ]:
# Будем сохранять loss во время обучения
# и рисовать график в режиме реального времени
train_loss_set = []
train_loss = 0


# Обучение
model.train()


for step, batch in enumerate(train_dataloader):
    # добавляем батч для вычисления на GPU
    batch = tuple(t.to(device) for t in batch)
    # Распаковываем данные из dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    optimizer.zero_grad()
    
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

    train_loss_set.append(loss[0].item())  
    
    # Backward pass
    loss[0].backward()
    
    # Обновляем параметры и делаем шаг используя посчитанные градиенты
    optimizer.step()

    # Обновляем loss
    train_loss += loss[0].item()
    
    # Рисуем график
    clear_output(True)
    plt.plot(train_loss_set)
    plt.title("Training loss")
    plt.xlabel("Batch")
    plt.ylabel("Loss")
    plt.show()
    
print("Loss на обучающей выборке: {0:.5f}".format(train_loss / len(train_dataloader)))


# Валидация
model.eval()

valid_preds, valid_labels = [], []

for batch in validation_dataloader:   
    # добавляем батч для вычисления на GPU
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Перемещаем logits и метки классов на CPU для дальнейшей работы
    logits = logits[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    batch_preds = np.argmax(logits, axis=1)
    batch_labels = np.concatenate(label_ids)     
    valid_preds.extend(batch_preds)
    valid_labels.extend(batch_labels)

print("Процент правильных предсказаний на валидационной выборке: {0:.2f}%".format(
    accuracy_score(valid_labels, valid_preds) * 100
))

In [ ]:
print("Процент правильных предсказаний на валидационной выборке: {0:.2f}%".format(
    accuracy_score(valid_labels, valid_preds) * 100))

# Оценка качества на отложенной выборке

In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(
    input_ids,
    maxlen=100,
    dtype="long",
    truncating="post",
    padding="post"
)

In [ ]:
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(test_gt)

prediction_data = TensorDataset(
    prediction_inputs,
    prediction_masks,
    prediction_labels
)

prediction_dataloader = DataLoader(
    prediction_data, 
    sampler=SequentialSampler(prediction_data),
    batch_size=32
)

In [ ]:
model.eval()
test_preds, test_labels = [], []

for batch in prediction_dataloader:
    # добавляем батч для вычисления на GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Распаковываем данные из dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Перемещаем logits и метки классов на CPU для дальнейшей работы
    logits = logits[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    batch_preds = np.argmax(logits, axis=1)
    batch_labels = np.concatenate(label_ids)  
    test_preds.extend(batch_preds)
    test_labels.extend(batch_labels)

In [ ]:
acc_score = accuracy_score(test_labels, test_preds)
print('Процент правильных предсказаний на отложенной выборке составил: {0:.2f}%'.format(
    acc_score*100
))

In [ ]:
print('Неправильных предсказаний: {0}/{1}'.format(
    sum(test_labels != test_preds),
    len(test_labels)
))